### Load the dataset

In [1]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, spa_path, wayuu_path):
        with open(spa_path, 'r', encoding='utf-8') as f:
            self.spa_lines = [line.strip() for line in f if line.strip()]

        with open(wayuu_path, 'r', encoding='utf-8') as f:
            self.wayuu_lines = [line.strip() for line in f if line.strip()]

    def __len__(self):
        return len(self.spa_lines)

    def __getitem__(self, idx):
        spa = self.spa_lines[idx]
        wayuu = self.wayuu_lines[idx]
        
        return spa, wayuu
    
spanish_val_file = 'datasets/dev.es.txt'
wayuu_val_file = 'datasets/dev.guc.txt'

# Load the dataset
dataset = TextDataset(spanish_val_file, wayuu_val_file)

### Eval function

In [2]:
import torch
import sacrebleu
from tqdm import tqdm
from vllm import SamplingParams

def get_rewards_translation(generations, correct_translations):

    bleu = sacrebleu.BLEU(effective_order = True)
    def get_bleu_score(sample, correct_translation):
        # Compute bleu score for each sample. 
        # Bleu score normalized to [0, 1]
        return bleu.sentence_score(sample, 
                                   [correct_translation]
                                   ).score

    answer_bleu_scores = [
        get_bleu_score(sample, translation)
        for sample, translation in zip(generations, correct_translations)
    ]
    
    return answer_bleu_scores

translate_prompt_template_tool="""Translate the following Spanish text into Wayuunaiki.
Begin by identifying any words or phrases you're unsure how to translate. Then, you may look up those words using the dictionary tool by wrapping the Spanish word in <spa_to_wayuu> and </spa_to_wayuu>,
and doind that for every unknown word. The dictionary will return matches enclosed in <matches> and </matches>. You can use the dictionary as many times as necessary.
Once you have all the information you need, provide the final translation enclosed in <answer> and </answer>. For example: <answer> xxx </answer>.

Spanish text: {}"""
def generate_batch_completion(model, tokenizer, prompts: list, actions_num=1, custom_prompt_template=None, **kwargs):
    prompt_template = custom_prompt_template if custom_prompt_template else translate_prompt_template_tool
    batch = [[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_template.format(prompt)}
    ] for prompt in prompts]
    texts = tokenizer.apply_chat_template(
        batch,
        tokenize=False,
        add_generation_prompt=True,
    )

    default_sampling_args = {
        'max_new_tokens': 512,
        'temperature': 0.8,
        'top_p': 0.95,
    }
    default_sampling_args.update(kwargs)

    model_inputs = tokenizer(texts)

    inputs = model_inputs.input_ids
    dones = [False] * len(prompts)
    prompt_length = [len(input_ids) for input_ids in inputs]
    mask = [[1] * len(input_ids) for input_ids in inputs]
    responses = [""] * len(prompts)
    tools_enabled = kwargs.get('tools', [])
    tools_enabled = [] if tools_enabled is None else tools_enabled
    stop_tokens = [tool['end_token'] for tool in tools_enabled]
    tool_used = [False] * len(prompts)
    how_many_tool_calls = [0] * len(prompts)
    for action_step in range(actions_num + 1 if len(tools_enabled) > 0 else 1):
        sampling_params = SamplingParams(temperature=default_sampling_args["temperature"], top_p=default_sampling_args['top_p'], top_k=-1, max_tokens=default_sampling_args['max_new_tokens'],
            stop=stop_tokens)
        outputs = model.generate(prompt_token_ids=inputs, sampling_params=sampling_params, lora_request=kwargs['lora_request'], use_tqdm=False)

        for j, output in enumerate(outputs):
            if dones[j]:
                continue
            
            for tool in tools_enabled:
                if output.outputs[0].stop_reason == tool['end_token'] and tool['start_token'] in output.outputs[0].text:
                    api_args = output.outputs[0].text.split(tool['start_token'])[1].strip()
                    api_result = tool['api'](api_args)
                    # responses[j] += f"{tool['start_token']} " + api_args + f" {tool['end_token']}" + api_result
                    responses[j] += output.outputs[0].text + f"{tool['end_token']}" + api_result
                    api_result_tokens = tokenizer.encode(api_result, return_tensors=None)
                    inputs[j] += list(output.outputs[0].token_ids) + api_result_tokens

                    tool_used[j] = True
                    how_many_tool_calls[j] += 1
                    break # Only one tool can be used at a time
            if output.outputs[0].finish_reason == "stop" and output.outputs[0].stop_reason is None:
                responses[j] += output.outputs[0].text
                inputs[j] += list(output.outputs[0].token_ids)
                dones[j] = True
            elif output.outputs[0].stop_reason not in stop_tokens:
                print(f"Unexpected finish reason: {output.outputs[0].finish_reason} {output.outputs[0].stop_reason}")
                responses[j] += tokenizer.eos_token
                inputs[j] += [tokenizer.eos_token_id]
                dones[j] = True
        
        if all(dones):
            break

    return responses, tool_used, how_many_tool_calls

import re

def extract_answer(response, transform_fn = lambda x: x, nan_val = None)->str|None:
    ans = re.match('.*?<answer>(.*?)</answer>\.?\s*$', response, re.DOTALL|re.MULTILINE)
    if ans:
        try:
            return transform_fn(ans[1].strip())
        except:
            return nan_val
    return nan_val

def evaluate_model(model, tokenizer, dataloader, actions_num=1, lora_request=None, tools=None, custom_prompt_template=None):
    sum_bleu = 0
    num_samples = 0
    tools_used_in_total = 0
    calls_per_sample = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            inputs, targets = batch

            # Generate translations
            outputs, tools_used, how_many_tool_calls = generate_batch_completion(model, tokenizer, inputs, actions_num=actions_num, lora_request=lora_request, tools=tools, temperature=0, top_p=1, max_new_tokens=768, custom_prompt_template=custom_prompt_template)

            tools_used_in_total += sum(tools_used)
            calls_per_sample += sum(how_many_tool_calls)

            generated_translations = [
                extract_answer(output, transform_fn=lambda x: x.strip(), nan_val='')
                for output in outputs
            ]
            # Calculate BLEU scores
            bleu_scores = get_rewards_translation(generated_translations, targets)
            
            sum_bleu += sum(bleu_scores)
            num_samples += len(bleu_scores)
    avg_bleu = sum_bleu / num_samples if num_samples > 0 else 0
    tools_used_avg = tools_used_in_total / num_samples
    calls_per_sample_avg = calls_per_sample / tools_used_in_total if tools_used_in_total > 0 else 0
    return avg_bleu, tools_used_avg, calls_per_sample_avg

/home/manuel/projects/rl_translator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-26 12:34:20,724	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


### tools

In [3]:
def spa_to_wayu_dictionary(spanish_word, max_matches=5):
    dictionary_path = 'assets/spanish_to_wayuunaiki_short.csv'

    with open(dictionary_path, 'r', encoding='utf-8') as f:
        all_matches = []
        line = f.readline()
        while line != '' and len(all_matches) < max_matches:
            data = line.strip().split(',')
            if re.search(rf'\b{re.escape(spanish_word)}\b', data[0], re.IGNORECASE):
                all_matches.append(data)
            line = f.readline()

    if len(all_matches) > 0:
        result = " <matches> " + '\n'.join(f'{spa}: {wayuu}' for spa, wayuu in all_matches) + " </matches>"
        # print(f'CORRECT USE OF SPA_TO_WAYU TOOL. Word: {spanish_word}, Result: {result}')
    else:
        result = " <matches> No matches found </matches>"
        # print(f'NO_MATCHES SPA_TO_WAYU TOOL. Word: {spanish_word}')

    return result

TOOLS = [
    {
        'name': 'spa_to_wayu',
        'description': 'A tool that translates a word from Spanish to Wayuunaiki.',
        'api': spa_to_wayu_dictionary,
        'start_token': '<spa_to_wayuu>',
        'end_token': '</spa_to_wayuu>',
    }
]

## Base Model

In [4]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

translate_prompt_template="""Translate the following Spanish text into Wayuunaiki. Provide the final translation enclosed in <answer> and </answer>. For example: <answer> xxx </answer>.
Spanish text: {}"""

base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2060,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, custom_prompt_template=translate_prompt_template)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-26 12:34:23,894] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-26 12:34:25 __init__.py:207] Automatically detected platform cuda.
INFO 05-26 12:34:29 config.py:549] This model supports multiple tasks: {'generate', 'reward', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
INFO 05-26 12:34:29 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2060, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.94it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.94it/s]


INFO 05-26 12:34:31 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-26 12:34:31 punica_selector.py:18] Using PunicaWrapperGPU.


...

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
WARNING 05-26 12:56:12 scheduler.py:1754] Sequence group 6569 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_pa

 99%|█████████▉| 103/104 [21:40<00:12, 12.47s/it]

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


100%|██████████| 104/104 [21:48<00:00, 12.58s/it]

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Average BLEU score: 0.1028
Average tools used: 0.0000
Average calls per sample: 0.0000


## Base Model + tools

In [4]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest


base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2060,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, actions_num=4, lora_request=None, tools=TOOLS)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-26 12:08:59,709] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-26 12:09:01 __init__.py:207] Automatically detected platform cuda.
INFO 05-26 12:09:06 config.py:549] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 05-26 12:09:06 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2060, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.32it/s]


INFO 05-26 12:09:08 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-26 12:09:08 punica_selector.py:18] Using PunicaWrapperGPU.


...

WARNING 05-26 12:29:40 scheduler.py:1754] Sequence group 30578 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=3301
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


 99%|█████████▉| 103/104 [20:27<00:12, 12.31s/it]

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


100%|██████████| 104/104 [20:35<00:00, 11.88s/it]

Average BLEU score: 0.0930
Average tools used: 0.3812
Average calls per sample: 1.0107


## Model with SFT without tools

In [4]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest


# def load_model(model_name_or_path, peft_model_id):
#     # Load the base model
#     model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", torch_dtype='auto')
#     tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

#     # Load the PEFT model
#     peft_model = PeftModel.from_pretrained(model, peft_model_id)
    
#     return peft_model, tokenizer


vllm_lora_adapter = 'models/sft_base_qwen_no_tools'
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2060,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, actions_num=4, lora_request=LoRARequest('adapter', 1, vllm_lora_adapter), tools=TOOLS)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-23 21:17:03,093] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-23 21:17:04 __init__.py:207] Automatically detected platform cuda.
INFO 05-23 21:17:09 config.py:549] This model supports multiple tasks: {'embed', 'score', 'reward', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 05-23 21:17:09 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2060, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.30it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.30it/s]


INFO 05-23 21:17:11 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-23 21:17:11 punica_selector.py:18] Using PunicaWrapperGPU.


...

Unexpected finish reason: length None


 99%|█████████▉| 103/104 [32:09<00:14, 14.71s/it]

Unexpected finish reason: length None


100%|██████████| 104/104 [32:16<00:00, 18.62s/it]

Average BLEU score: 8.2227
Average tools used: 0.0000
Average calls per sample: 0.0000


### Model with SFT with tools

In [4]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest


# def load_model(model_name_or_path, peft_model_id):
#     # Load the base model
#     model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", torch_dtype='auto')
#     tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

#     # Load the PEFT model
#     peft_model = PeftModel.from_pretrained(model, peft_model_id)
    
#     return peft_model, tokenizer


vllm_lora_adapter = 'models/sft_base_qwen2'
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2060,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, actions_num=4, lora_request=LoRARequest('adapter', 1, vllm_lora_adapter), tools=TOOLS)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-21 14:53:23,603] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-21 14:53:24 __init__.py:207] Automatically detected platform cuda.
INFO 05-21 14:53:28 config.py:549] This model supports multiple tasks: {'generate', 'reward', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 05-21 14:53:28 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fal

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.57it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.57it/s]


INFO 05-21 14:53:30 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-21 14:53:30 punica_selector.py:18] Using PunicaWrapperGPU.


...

WARNING 05-21 15:13:00 scheduler.py:1754] Sequence group 32411 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1001
Unexpected finish reason: length None
Unexpected finish reason: length None


 99%|█████████▉| 103/104 [19:35<00:10, 10.11s/it]

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


100%|██████████| 104/104 [19:43<00:00, 11.38s/it]

Average BLEU score: 13.2998
Average tools used: 0.9025
Average calls per sample: 2.1136


## Model with SFT with tools + RL with tools

In [4]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest


vllm_lora_adapter = 'models/best_policy_model11'
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2048,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=48, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, actions_num=4, lora_request=LoRARequest('adapter', 1, vllm_lora_adapter), tools=TOOLS)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-26 10:49:48,828] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-26 10:49:49 __init__.py:207] Automatically detected platform cuda.
INFO 05-26 10:49:54 config.py:549] This model supports multiple tasks: {'generate', 'reward', 'score', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 05-26 10:49:54 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.04s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.04s/it]


INFO 05-26 10:49:57 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-26 10:49:57 punica_selector.py:18] Using PunicaWrapperGPU.


...

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


 99%|█████████▉| 138/139 [35:54<00:15, 15.58s/it]

Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None
Unexpected finish reason: length None


100%|██████████| 139/139 [35:56<00:00, 15.52s/it]

Average BLEU score: 21.5450
Average tools used: 0.9998
Average calls per sample: 3.7474


### Model with SFT no tools + RL

In [5]:
# from pretrained peft model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest


vllm_lora_adapter = 'models/best_policy_model10'
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

inference_engine = LLM(
    model=base_model_name,
    enable_lora=True,
    max_lora_rank=64,
    max_loras=1,
    gpu_memory_utilization=0.2,
    # enable_prefix_caching=True,
    swap_space=6,
    scheduling_policy="fcfs",
    dtype=torch.bfloat16,
    max_model_len=2048,
    # enable_sleep_mode=True,
    )

dataloader = DataLoader(dataset, batch_size=48, shuffle=True)

# Evaluate the model
avg_bleu, tools_used_avg, calls_per_sample_avg = evaluate_model(inference_engine, tokenizer, dataloader, actions_num=4, lora_request=LoRARequest('adapter', 1, vllm_lora_adapter), tools=TOOLS)
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average tools used: {tools_used_avg:.4f}")
print(f"Average calls per sample: {calls_per_sample_avg:.4f}")

[2025-05-26 11:29:47,133] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 05-26 11:29:49 __init__.py:207] Automatically detected platform cuda.
INFO 05-26 11:29:53 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 05-26 11:29:53 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]


INFO 05-26 11:29:56 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 05-26 11:29:56 punica_selector.py:18] Using PunicaWrapperGPU.


...

Unexpected finish reason: length None


 99%|█████████▉| 138/139 [11:14<00:04,  4.25s/it]

Unexpected finish reason: length None


100%|██████████| 139/139 [11:14<00:00,  4.86s/it]

Average BLEU score: 8.1556
Average tools used: 0.0000
Average calls per sample: 0.0000
